# <font color='green'><b> SIFT operator </b></font>


### Credits: Hands-on Image Processing with Python, Chapter 7 - Author: Sandipan Dey


In [ ]:
#@title ▶️ Base dir setup
import os, sys

# check if hosted (Google VM) or running on local server
if 'google.colab' in sys.modules:
  #@markdown Google Drive root folder - hosted by Google VM (adapt to your local paths)
  from google.colab import drive
  drive.mount('/content/drive', force_remount=False)
  base_dir = 'CV/' #@param {type: "string"}
  base_dir  = os.path.join('/content/drive/MyDrive/', base_dir)
  #!pip install pillow  --upgrade
  img_dir = 'data/img/'
  vid_dir = 'data/video/'
  out_dir = 'output/'
  
  # move to base_dir 
  os.chdir(base_dir)
else:
  #@markdown Path to local folder on PC (adapt to your local paths)
  img_dir = '../data/img/'
  out_dir = '../data/output/'

 

print("Current dir:", os.getcwd())

In [ ]:
import numpy as np
import cv2 
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import ndimage
from scipy.spatial import distance
from scipy.spatial import distance_matrix
import pickle
import math
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
from skimage.color import rgb2gray
from skimage import img_as_float, img_as_ubyte

cv2.__version__

In [ ]:
# load an example image 
img = imread(img_dir + 'monalisa.jpg')# 'flowers.png') #'flowers.jpg')  
gray = img_as_ubyte(rgb2gray(img))

# show image
plt.figure(figsize=(6,6))
plt.imshow(img)
plt.show()

## <font color='green'><b>SIFT in OpenCV</b></font>


So now let’s see SIFT functionalities available in OpenCV. Let’s start with keypoint detection and draw them. First we have to construct a SIFT object. We can pass different parameters to it which are optional and they are well explained in docs.

https://docs.opencv.org/master/da/df5/tutorial_py_sift_intro.html


**1. create the SIFT object:**
 
`sift = cv2.SIFT_create()` https://docs.opencv.org/4.4.0/d7/d60/classcv_1_1SIFT.html

It has several parameters, but interesting to us:


- *nfeatures:*	The number of best features to retain (if 0, it returns all the kp it found). The features are ranked by their scores (measured in SIFT algorithm as the local contrast)
- *nOctaveLayers:*	The number of layers in each octave. 3 is the value used in D. Lowe paper. The number of octaves is computed automatically from the image resolution.

- *contrastThreshold:*	The contrast threshold used to filter out weak features in semi-uniform (low-contrast) regions. The larger the threshold, the less features are produced by the detector.
- *edgeThreshold:*	The threshold used to filter out edge-like features. Note that the its meaning is different from the contrastThreshold. The larger the edgeThreshold, the less features are filtered out (more features are retained).

- *sigma:*	The sigma of the Gaussian applied to the input image at the octave #0. If your image is captured with a weak camera with soft lenses, you might want to reduce the number.


In [ ]:
#DEFAULT PARAMETERS VALUES (EQUIVALENT TO CALL sift = cv2.SIFT_create() ):
sift = cv2.SIFT_create(nfeatures = 0, nOctaveLayers = 3, contrastThreshold = 0.1, edgeThreshold = 10, sigma = 2)	 

**2. Keypoint detection**

`kp = sift.detect(gray,None)`

`kp` is the list of the detected keypoints, eqch one with the field; 
- *pt*:	x & y coordinates of the keypoint

- *size*:	keypoint diameter

- *angle*:	keypoint orientation

- *response*:	keypoint detector response on the keypoint (that is, strength of the keypoint)

- *octave*:	pyramid octave in which the keypoint has been detected

- *class_id*:	object id

In [ ]:
def print_KP_info(kp, i):
  print('Keypoint coordinates: '+ str(kp[i].pt)) 
  print('Keypoint size: '+ str(kp[i].size))  
  print('Keypoint orientation: '+ str(kp[i].angle))  
  print('Keypoint response: '+ str(kp[i].response))   

In [ ]:
#detection
kp = sift.detect(gray, None)
print('Detected KP: '+ str(len(kp)));

KP_i= 0
print_KP_info(kp, KP_i)

##### <font color='green'><b>EXERCISE 1: </b></font>
Modify the contrastThreshold, the edgeThreshold, and the sigma parameters, and observe the variation in keypoint detection

##### <font color='green'><b>EXERCISE 2: </b></font>
 
Given the list of keypoint, detect the one with the strongest response, and print all the corresponding information using the function `print_KP_info(kp, i)`

In [ ]:
#TO DO

In [ ]:
#SOLUTION 1 TO EX 2

#detect the keypoint with the strongest response
max_response = float('-Inf')
max_pos = -1;

for i in range(len(kp)):
  val = kp[i].response;
  if val > max_response:
      max_response = val;
      max_pos = i;

print_KP_info(kp, max_pos)


In [ ]:
#SOLUTION 2 TO EX 2 (BETTER)
val = [kp[i].response for i in range(len(kp))];
kp_max_resp = np.argmax(val)

print_KP_info(kp, kp_max_resp)

##### <font color='green'><b>EXERCISE 3: </b></font>
 
Given the list of keypoint, detect the one with the largest size, and print all the corresponding information using the function `print_KP_info(kp, i)`

In [ ]:
#TO DO
 

In [ ]:
#SOLUTION TO EX 3
 
val= [kp[i].size for i in range(len(kp))];
kp_max_size = np.argmax(val)

print_KP_info(kp, kp_max_size)

**3. Keypoint descriptor**

`keypoints, des = sift.compute(image, keypoints)`
 
Parameters:
- *image*:	Image.
- *keypoints*:	[List] Input collection of keypoints. Keypoints for which a descriptor cannot be computed are removed. Sometimes new keypoints can be added, for example: SIFT duplicates keypoint with several dominant orientations (for each orientation).
- *descriptors*:	[array of array] Computed descriptors. In the second variant of the method descriptors[i] are descriptors computed for a keypoints[i]. Row j is the keypoints (or keypoints[i]) is the descriptor for keypoint j-th keypoint.


In [ ]:
# compute the keypoint description 
kp, des = sift.compute(gray, kp)

print("Descrpitor dimension: " + str(np.squeeze(des[0].shape)))
print(des[0])
plt.plot(des[0].flatten())

#OR
#detection and description in one step
kp, des = sift.detectAndCompute(gray,None);

 

In [ ]:
#draw the keypoints on an image
imgKP=cv2.drawKeypoints(gray,kp,img,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
#img=cv2.drawKeypoints(gray,kp,img) #simple visualization

plt.figure(figsize=(30,10))
plt.imshow(imgKP, cmap='gray', vmin=0, vmax=255)
plt.show()

##### <font color='green'><b>EXERCISE 4: </b></font>
 
Given the keypoint with the maximum response (keypoint: 'kp_max_resp') plot only it using opportunely the function cv2.drawKeypoints.
do the same for the kp with max size


In [ ]:
#TO DO




In [ ]:
#SOLUTION TO EX 4

imgKP_strongest = cv2.drawKeypoints(gray,[kp[kp_max_resp]],img,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
#imgKP_strongest = cv2.drawKeypoints(gray,[kp[kp_max_resp]],None,color=(0,255,0))
plt.figure(figsize=(20,10))
plt.imshow(imgKP_strongest)
plt.show()